# SQL query from table names - Continued

In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [8]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [10]:
context = [ {'role':'system', 'content':"""
 create table movies(
        ID_movie INT primary key,
        title VARCHAR,
        release_date DATE,
        genre VARCHAR);
    /* 3 example rows
    select * from movies limit 3;
    ID_movie    title                 release_date    genre
    101         Inception             2010-07-16     Sci-Fi
    102         Titanic               1997-12-19     Romance
    103         The Dark Knight       2008-07-18     Action
    */

    create table cost(
        ID_movie INT,
        production_cost FLOAT,
        promo_cost FLOAT,
        total_cost FLOAT as (production_cost + promo_cost),
        foreign key (ID_movie) references movies(ID_movie));
    /* 3 example rows
    select * from cost limit 3;
    ID_movie    production_cost    promo_cost    total_cost
    101         160000000          100000000     260000000
    102         200000000          50000000      250000000
    103         185000000          150000000     335000000
    */

    create table entry(
        ID_entry INT primary key,
        ID_movie INT,
        country VARCHAR,
        tickets_sold INT,
        ticket_price FLOAT,
        revenue FLOAT as (tickets_sold * ticket_price),
        foreign key (ID_movie) references movies(ID_movie));
    /* 3 example rows
    select * from entry limit 3;
    ID_entry    ID_movie    country    tickets_sold    ticket_price    revenue
    1           101         USA        30000000        10.0           300000000
    2           102         France     20000000        12.0           240000000
    3           103         USA        40000000        9.5            380000000
    */
"""} ]



In [11]:
# FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the tables provided above.
 Question: What is the total revenue generated by all movies?
SELECT SUM(revenue) AS total_revenue
FROM entry;

 Question: Which movie has the highest production cost?
SELECT m.title
FROM movies m
JOIN cost c ON m.ID_movie = c.ID_movie
ORDER BY c.production_cost DESC
LIMIT 1;

 Question: How many tickets were sold for "Inception" in the USA?
SELECT e.tickets_sold
FROM entry e
JOIN movies m ON e.ID_movie = m.ID_movie
WHERE m.title = 'Inception' AND e.country = 'USA';

 Question: List the top 3 countries with the highest average ticket price for all movies.
SELECT country, AVG(ticket_price) AS avg_price
FROM entry
GROUP BY country
ORDER BY avg_price DESC
LIMIT 3;
"""
})

In [12]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [13]:
#new
context_user = context.copy()
print(return_CCRMSQL("""which movie sold the most tickets in France ?""", context_user))

SELECT m.title
FROM movies m
JOIN entry e ON m.ID_movie = e.ID_movie
WHERE e.country = 'France'
ORDER BY e.tickets_sold DESC
LIMIT 1;


In [14]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("who are the top 3 highest paid employees ? ", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 3;
```

This SQL query selects the names and salaries of the employees by joining the "employees" and "salary" tables on the ID_usr field. It then orders the results by salary in descending order and limits the output to the top 3 highest paid employees.


In [15]:
#new
print(return_CCRMSQL("which contry generated the most revenue for all the films", context_user))

```sql
SELECT e.country, SUM(e.revenue) AS total_revenue
FROM entry e
GROUP BY e.country
ORDER BY total_revenue DESC
LIMIT 1;
```


In [16]:
#old
print(return_CCRMSQL("what is the degree of the lowest paid employee ?", old_context_user))

This is your SQL:
```sql
SELECT MIN(educational_level) AS lowest_degree
FROM studies
```

This SQL query selects the minimum value of the educational level from the "studies" table, which represents the degree of the lowest paid employee.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?